In [24]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,confusion_matrix, accuracy_score, make_scorer, f1_score,precision_score,recall_score, plot_confusion_matrix
nltk.download('rslp')
nltk.download('stopwords')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\cfpc2\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cfpc2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
dataset = pd.read_csv("datasets/reviews.csv")

In [12]:
dataset.head(5)

,Unnamed: 0,order_id,review_id,review_score,review_comment_message
0,3,658677c97b385a9be170737859d3511b,e64fb393e7b32834bb789ff8bb30750e,1,Recebi bem antes do prazo estipulado.
1,4,8e6bfb81e283fa7e4f11123a3fb894f1,f7c4243c7fe1938f181bec41a392bdeb,1,Parabéns lojas lannister adorei comprar pela I...
2,9,b9bf720beb4ab3728760088589c62129,8670d52e15e00043ae7de4c01cc2fe06,0,aparelho eficiente. no site a marca do aparelh...
3,12,9d6f15f95d01e79bd1349cc208361f09,4b49719c8a200003f700d3d986ea1a19,0,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,15,e51478e7e277a83743b6f9991dbfa3fb,3948b09f7c818e2d86c9a546758b2335,1,"Vendedor confiável, produto ok e entrega antes..."


In [13]:
stopword = stopwords.words("portuguese")
stem = RSLPStemmer()
vectorizer = TfidfVectorizer()
def clear(review):
  review = review.lower()
  # remove pula de linha 
  review = re.sub('\n', ' ', review)        
  review = re.sub('\r', ' ', review)

  # remove numero 
  review = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', ' #numero ', review)

  # remove caracters especiais 
  review = re.sub(r'R\$', ' ', review)
  review = re.sub(r'\W', ' ', review)
  review = re.sub(r'\s+', ' ', review)

  # remove links 
  urls = re.findall('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', review)
  if len(urls) > 0:
      for url in urls:
          for link in url:
              review = review.replace(link, '')
      review = review.replace(':', '')
      review = review.replace('/', '')
  return review
reviews = dataset["review_comment_message"]
reviews = reviews.apply(lambda review: clear(review))
reviews = reviews.apply(lambda review: word_tokenize(review))
reviews = reviews.apply(lambda words_review: [word for word in words_review if word not in stopword])
reviews = reviews.apply(lambda words_review: [stem.stem(word) for word in words_review ])
reviews = reviews.apply(lambda words_review: " ".join(words_review))
reviews = vectorizer.fit_transform(reviews)
reviews


<40874x7920 sparse matrix of type '<class 'numpy.float64'>'
	with 277461 stored elements in Compressed Sparse Row format>

In [14]:
X = reviews.copy()
y = dataset["review_score"].copy()
y = np.array(y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 199)

In [16]:
lr = LogisticRegression(random_state=199)
lr.fit(X_train, y_train)
predicted_y =  lr.predict(X_test)

C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

          -1       0.74      0.85      0.79      3208
           0       0.56      0.25      0.35      2918
           1       0.77      0.92      0.84      6137

    accuracy                           0.74     12263
   macro avg       0.69      0.67      0.66     12263
weighted avg       0.72      0.74      0.71     12263



In [32]:
parameters = dict()
parameters["penalty"] = ["l1","l2","elasticnet","none"]
parameters["solver"] = ["newton-cg", "lbfgs","liblinear", "sag","saga"]
parameters["C"] = np.logspace(-4, 4, 20)

In [33]:
lrgs = LogisticRegression(random_state=199)

In [34]:
search = GridSearchCV(lrgs, parameters, scoring="accuracy", cv=10, verbose=3, refit=True,n_jobs=-4)

In [35]:
result = search.fit(X_train, y_train)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1800 fits failed out of a total of 4000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\cfpc2\anaconda3\envs\rv\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _ch

In [36]:
best_model = result.best_estimator_
predicted_y = best_model.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

          -1       0.74      0.86      0.80      3208
           0       0.58      0.24      0.34      2918
           1       0.77      0.92      0.84      6137

    accuracy                           0.75     12263
   macro avg       0.70      0.68      0.66     12263
weighted avg       0.72      0.75      0.71     12263



In [37]:
print(result.best_estimator_)
print(search.best_params_)

LogisticRegression(C=0.615848211066026, random_state=199, solver='saga')
{'C': 0.615848211066026, 'penalty': 'l2', 'solver': 'saga'}
